# Игра с клетками

В этой задаче поиграем в игру на поле $m\times n$, клетки которого могут быть в двух состояниях: «горящие» или «потухшие». При нажатии клетка и её четыре соседа по диагонали меняют состояние. Цель игры — потушить все клетки.

![Игра с квадратами](https://imgur.com/Qs9hGN1.png)

Игра обманчиво проста, но победить в ней не так-то легко. Попробуйте [поиграть.](https://htmlpreview.github.io/?https://github.com/dkosolobov/squaregame/blob/main/squaregame.html)

В этой задаче вам надо разработать программу, которая с помощью систем уравнений в поле $Z_2$ по заданному игровому полю определяет, можно ли победить и как именно.

**1. Система $4 \times 4$.** Допишите функцию, которая получает на вход $4\times 4$ матрицу $A$ над полем $\mathbb{Z}_2$ и столбец $b$ размера 4 (т.е. $A \in \mathbb{Z}_2^{4\times 4}$ и $b \in \mathbb{Z}_2^4$) и возврашает `True`, если система $Ax=b$ имеет решение в поле $\mathbb{Z}_2$, и `False` в противном случае.  Можно пропустить это задание, если вы выполните следующее. Техническое напоминание: код обмена строк матрицы выглядит как `A[[i,j]]=A[[j,i]]`.

In [61]:
def is_4x4_system_solvable(A, b):
    '''Разрешимость системы Ax = b над полем {0,1}, где A — матрица 4х4'''
    A = A.copy() 
    b = b.copy()
    return is_nxn_system_solvable(A, b)

**2. Система $n \times n$.** Допишите функцию, которая получает на вход $n\times n$ матрицу $A$ над полем $\mathbb{Z}_2$ и столбец $b$ размера $n$ (т.е. $A \in \mathbb{Z}_2^{n\times n}$ и $b \in \mathbb{Z}_2^n$) и возврашает `True`, если система $Ax=b$ имеет решение в поле $\mathbb{Z}_2$, и `False` в противном случае.

In [62]:
def is_nxn_system_solvable(A, b):
    '''Разрешимость системы Ax = b над полем {0,1}, где A — матрица nхn'''
    A = A.copy()
    B = np.column_stack((A, b))

    if get_rank(A) == get_rank(B):   
        return True                                
    return False              

def gauss(X, for_solution):
    matrix = X.copy()
    rows = matrix.shape[0]                       
    
    for i in range(0, rows):
        if matrix[i, i] == 0:
            row_with_no_zero = get_row_with_no_zero(matrix, i)
            if(row_with_no_zero == -1):
                continue
            matrix[[i, row_with_no_zero]] = matrix[[row_with_no_zero, i]]

        for j in range(i+1,rows):
            if(matrix[j, i] == 0):
                continue
            row_addition(matrix, j, i)

    for i in range(rows-1,-1,-1):
        for j in range(i-1,-1,-1):
            if(matrix[j,i] == 0):
                continue
            row_addition(matrix, j, i)
            
    if for_solution:
        return matrix[:, matrix.shape[1] - 1]
    else:
        return matrix

def get_row_with_no_zero(X, column):
    columns = len(X)
    for i in range(column+1, columns):
        if(X[i, column] != 0):
            return i
    return -1

'''Прибавляет строку j к строке i в кольце вычета 2'''
def row_addition(X, i, j):
    for k in range(0, X.shape[1]):
        X[i, k] = (X[i, k] + X[j, k]) % 2

def get_rank(X):
    zero_rows = 0
    matrix = gauss(X, False)
    rows = matrix.shape[0]
    
    for row in range(rows):
        if sum(matrix[row]) == 0:
            zero_rows += 1
            
    return rows - zero_rows

**3. Разрешимость игры.** Допишите функцию, которая возвращает `True`, если в заданной игре можно победить, и `False` в противном случае. Игровое поле представлено матрицей нулей и единиц размера $m \times n$, в которой ноль означает потухшую клетку, единица — горящую. Подсказка: с помощью `is_4x4_system_solvable` можно разрешать игровые поля размеров $2\times 2$, $4 \times 1$ и $1 \times 4$.

In [63]:
def game_solvable(GameField):
    '''Разрешимость игрового поля GameField[0:m,0:n] из нулей и единиц'''
    GameField = GameField.copy() # эту копию GameField можно модифицировать
    
    A = get_dependence_matrix(GameField)
    b = GameField.flatten()
            
    return is_nxn_system_solvable(A, b)

def cell_exist(x, y, m, n):
    return 0 <= x < m and 0 <= y < n

def dependence_matrix(GameField):                      
    [rows, columns] = GameField.shape                               
                                                           
    X = np.zeros([rows * columns, rows * columns])                          
    
    for row in range(rows):
        for column in range(columns):
            crd = column + columns * row                       
            X[crd][crd] = 1                            
            
            if cell_exist(row - 1, column - 1, rows, columns):  
                X[crd - columns - 1][crd] = 1                
                
            if cell_exist(row - 1, column + 1, rows, columns):
                X[crd - columns + 1][crd] = 1
                
            if cell_exist(row + 1, column - 1, rows, columns):
                X[crd + columns - 1][crd] = 1
                
            if cell_exist(row + 1, column + 1, rows, columns):
                X[crd + columns + 1][crd] = 1
                
    return X

**4. Решение игры.** Если в игре можно победить, то хочется знать как. Допишите функцию, возвращающую список клеток, которые надо нажать для победы, и возвращающую `None`, если победить нельзя. Клетки задаются парами чисел `[x,y]`; например, список `[[0,0], [0,2], [1,1]]`  означает, что надо нажать на левую верхнюю клетку, затем третью клетку в верхнем ряду, а затем вторую клетку во втором ряду.

In [64]:
def solve_game(GameField):
    '''Найти решение игрового поля GameField[0:m,0:n] из нулей и единиц'''
    GameField = GameField.copy()
    [m, n] = GameField.shape
    
    if not game_solvable(GameField):                  
        return None
    
    A = dependence_matrix(GameField)
    b = GameField.flatten()
            
    solution_matrix = gauss(np.column_stack((A, b)), True)  
    cells = []                                       
    
    for i in range(m * n):
        if solution_matrix[i] == 1:
            cells.append([i // n, i % n])
    
    return cells                      

## Тестирование

Осталось проверить, проходят ли ваши решения первоначальное тестирование. Выполните весь код (Cell -> Run All или Runtime -> Run All или другим способом, работающим в вашей среде) и посмотрите вердикт внизу страницы. В тестирующем коде разбираться не нужно!


In [65]:
import numpy as np
from itertools import chain, product

solvable_4x4 = ({'A': [[0,0,0,1],[0,1,0,1],[0,1,1,0],[1,0,1,0]], 'b':[0,1,0,1]},
    {'A':[[0,1,0,1],[1,0,0,1],[1,1,1,1],[0,0,1,1]],'b': [1,1,0,0]})
unsolvable_4x4 = ({'A':[[0,1,0,1],[1,0,0,1],[1,1,1,1],[0,0,1,1]],'b':[1,1,0,1]},
    {'A':[[1,0,1,0],[0,1,1,0],[1,1,0,0],[0,0,0,0]],'b':[1,1,1,1]},
    {'A':[[1,0,1,1],[0,0,1,0],[0,0,0,0],[1,0,0,1]],'b': [0,1,0,0]})
solvable_nxn = solvable_4x4 + ({'A':[[1]],'b':[1]},{'A':[[1]],'b':[0]},
    {'A':[[0]],'b':[0]},{'A':[[0,0,0],[0,1,1],[1,1,1]],'b':[0,1,1]},
    {'A':[[0,0,0,0,0],[0,0,0,0,1],[0,0,1,1,0],[0,0,0,1,0],[0,0,1,0,1]],'b':[0,0,1,1,0]},
    {'A':[[1,1,0,0,1],[0,1,0,0,0],[0,1,0,1,0],[0,0,0,1,0],[1,1,0,1,1]],'b':[0,0,1,1,1]})
unsolvable_nxn = unsolvable_4x4 + ({'A':[[0]], 'b':[1]}, 
    {'A':[[0,0,0],[0,1,1],[1,1,1]],'b':[1,1,1]},
    {'A':[[0,0,0,0,0],[0,0,0,0,1],[0,0,1,1,0],[0,0,0,1,0],[0,0,1,0,1]],'b':[0,0,1,1,1]})
solvable_games = ([[0]],[[1]],[[0,0]],[[1,1]],[[1],[1]], [[1,0],[0,1]], [[0],[1],[0],[1]],
    [[0,1],[0,1],[1,0]],[[1,1,1,1,0],[1,1,0,1,1],[1,0,1,0,0]], [[1, 1, 1], [0, 1, 1], [1, 1, 1]],
    [[0,1,0,1],[0,1,0,1],[1,1,1,1],[1,1,1,1],[0,1,0,1]], 
    [[0,0,0,0,1,1], [1,1,1,1,1,1], [1,1,1,1,0,0], [0,0,1,1,1,1], [1,1,1,1,1,1], [1,1,0,0,0,0]])
unsolvable_games = ([[0,1],[0,1]], [[1,1],[0,0]], [[1,0],[1,0]], [[0,1],[1,1]], [[0,1,1,1,1],[1,1,1,1,1]], 
    [[0,1,0,1],[0,1,0,1],[0,1,0,1],[0,1,0,1]],
    [[0,0,1,1,0],[0,1,1,1,1],[1,1,1,1,1],[1,0,0,1,1],[0,0,1,1,1]],
    [[0,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1]] )

def seq_solves_game(seq, game):
    game = game.copy()
    if seq is None:
        return False
    for i, j in seq:
        for si, sj in [(0,0), (1,1), (1,-1), (-1,1), (-1,-1)]:
            if 0 <= i + si < len(game) and 0 <= j + sj < len(game[0]):
                game[i + si, j + sj] ^= 1
    return all(all(cell == 0 for cell in row) for row in game)

def test_all():
    test_seq = chain((t for t in solvable_4x4 if not is_4x4_system_solvable(
                      np.array(t['A']), np.array(t['b']))),
                   (t for t in unsolvable_4x4 if is_4x4_system_solvable(
                      np.array(t['A']), np.array(t['b']))))
    test = next(test_seq, None)
    if test is not None:
        print('Первое задание не прошло тест:')
        print('A =\n', np.array(test['A']), '\nb =', np.array(test['b']))
    else:
        print('Первое задание прошло все тесты')
  
    test_seq = chain((t for t in solvable_nxn if not is_nxn_system_solvable(
                      np.array(t['A']), np.array(t['b']))),
                   (t for t in unsolvable_nxn if is_nxn_system_solvable(
                      np.array(t['A']), np.array(t['b']))))
    test = next(test_seq, None)
    if test is not None:
        print('Второе задание не прошло тест:')
        print('A =\n', np.array(test['A']), '\nb =', np.array(test['b']))
    else:
        print('Второе задание прошло все тесты')

    test = next(chain((t for t in solvable_games 
                     if not is_game_solvable(np.array(t))),
                    (t for t in unsolvable_games 
                     if is_game_solvable(np.array(t)))), None)
    if test is not None:
        print('Третье задание не прошло тест:')
        print(np.array(test))
    else:
        print('Третье задание прошло все тесты')
  
    test = next(chain((t for t in solvable_games 
                     if not seq_solves_game(solve_game(np.array(t)), np.array(t))),
                    (t for t in unsolvable_games 
                     if solve_game(np.array(t)) is not None)), None)  
    if test is not None:
        print('Четвёртое задание не прошло тест:')
        print(np.array(test))
    else:
        print('Четвёртое задание прошло все тесты')

In [66]:
test_all()

Первое задание прошло все тесты
Второе задание прошло все тесты
Третье задание прошло все тесты
Четвёртое задание прошло все тесты
